# Atomic Physics
Tables and figures associated with the atomic physics details

In [2]:
import numpy as np
from astropy.table import Table,Column
from astropy.io import ascii

from synthesizAR.atomic import EmissionModel

%matplotlib inline

In [3]:
em = EmissionModel.restore('/storage-home/w/wtb2/data/timelag_synthesis_v2/base_emission_model.json')

## Element Table
List elements, number of ions, total number of lines

In [27]:
by_element = {}
for i in em:
    el = i.atomic_symbol
    if el not in by_element:
        by_element[el] = [i]
    else:
        by_element[el].append(i)
by_element = {k: sorted(v, key=lambda x:x.atomic_number) for k,v in by_element.items()}

In [34]:
col1 = [e[0].atomic_symbol for e in sorted([v for _,v in by_element.items()], key=lambda x: x[0].atomic_number)]

In [36]:
col2 = [len(by_element[x]) for x in col1]

In [39]:
col3 = [sum([i.transitions.wavelength.shape[0] for i in by_element[x] if i.transitions is not None]) for x in col1]

In [41]:
t = Table([col1, col2, col3], names=('Element', 'Number of Ions', 'Number of Transitions'),
          meta={'name': 'CHIANTI Elements'})

In [45]:
caption = r"""Elements included in the intensity calculation. These 7 elements were selected as they are the dominant contributors in the
AIA passbands. We use every ion from every element and every transition from every ion. All of the atomic data is gathered from the 
CHIANTI atomic database.\label{tab:elements}
"""

In [47]:
ascii.write(t, output='tex/element_table.tex', format='aastex',caption=caption,overwrite=True)